In [75]:
import pickle
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

with open('../encoders/target_encoding.pkl', 'rb') as f:
    target_encoder = pickle.load(f)
with open('../encoders/one_hot_encoder.pkl', 'rb') as f:
    one_hot_encoder = pickle.load(f)
with open('../encoders/robus_scaler.pkl', 'rb') as f:
    robust_scaler = pickle.load(f)    
with open('../models/mejor_modelo_random_forest.pkl', 'rb') as f:
    modelo = pickle.load(f)

In [76]:
print(one_hot_encoder.categories_)

[array([1, 2, 3, 4, 5], dtype=object), array(['Female', 'Male'], dtype=object), array(['Healthcare Representative', 'Human Resources',
       'Laboratory Technician', 'Manager', 'Manufacturing Director',
       'Research Director', 'Research Scientist', 'Sales Executive',
       'Sales Representative'], dtype=object)]


In [77]:
cols_ordinales = ['Department', 'EducationField', 'MaritalStatus']
cols_nominales = ['Education', 'Gender', 'JobRole']
cols_escalar = ['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
                'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany',
                'YearsSinceLastPromotion', 'YearsWithCurrManager']

In [78]:
def realizar_prediccion(EnvironmentSatisfaction, WorkLifeBalance, JobInvolvement, JobSatisfaction,
                        Age, Gender, MaritalStatus, DistanceFromHome, Department, JobLevel, JobRole,
                        YearsAtCompany, YearsSinceLastPromotion, YearsWithCurrManager, Education,
                        EducationField, TrainingTimesLastYear, MonthlyIncome, PercentSalaryHike,
                        StockOptionLevel, NumCompaniesWorked, TotalWorkingYears, BusinessTravel, PerformanceRating):
    # Definir columnas
    cols_ordinales = ['Department', 'EducationField', 'MaritalStatus']
    cols_nominales = ['Education', 'Gender', 'JobRole']
    cols_escalar = ['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
                    'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany',
                    'YearsSinceLastPromotion', 'YearsWithCurrManager']

    # Crear DataFrame de entrada
    new_employee = pd.DataFrame({
        'EnvironmentSatisfaction': [EnvironmentSatisfaction],
        'JobSatisfaction': [JobSatisfaction],
        'WorkLifeBalance': [WorkLifeBalance],
        'Age': [Age],
        'Attrition': ["Yes"],
        'BusinessTravel': [BusinessTravel],
        'Department': [Department],
        'DistanceFromHome': [DistanceFromHome],
        'Education': [Education],
        'EducationField': [EducationField],
        'Gender': [Gender],
        'JobLevel': [JobLevel],
        'JobRole': [JobRole],
        'MaritalStatus': [MaritalStatus],
        'MonthlyIncome': [MonthlyIncome],
        'NumCompaniesWorked': [NumCompaniesWorked],
        'PercentSalaryHike': [PercentSalaryHike],
        'StockOptionLevel': [StockOptionLevel],
        'TotalWorkingYears': [TotalWorkingYears],
        'TrainingTimesLastYear': [TrainingTimesLastYear],
        'YearsAtCompany': [YearsAtCompany],
        'YearsSinceLastPromotion': [YearsSinceLastPromotion],
        'YearsWithCurrManager': [YearsWithCurrManager],
        'JobInvolvement': [JobInvolvement],
        'PerformanceRating': [PerformanceRating]})

    # Copiar DataFrame
    df_pred = new_employee.copy()

    # Escalar valores
    df_pred[cols_escalar] = robust_scaler.transform(df_pred[cols_escalar])
    display(df_pred)
    # Convertir a 'object' todas las columnas excepto las excluidas
    df_pred = df_pred.astype({col: 'object' for col in new_employee.columns if col not in cols_escalar})
    
    # Realizar OneHot Encoding
    # Aplicar codificación one-hot
    trans_one_hot = one_hot_encoder.fit_transform(df_pred[cols_nominales])
    oh_df = pd.DataFrame(trans_one_hot, columns=one_hot_encoder.get_feature_names_out(cols_nominales))
    
    # Combinar el DataFrame original con el DataFrame codificado
    df_pred = pd.concat([df_pred.reset_index(drop=True), oh_df.reset_index(drop=True)], axis=1)

    # Eliminar las columnas originales
    df_pred.drop(columns=cols_nominales, inplace=True)
    display(df_pred)

    df_hot['BusinessTravel'] = df_hot['BusinessTravel'].map({'Non-Travel': 1, 'Travel_Rarely': 2, 'Travel_Frequently': 3})
    df_hot['Attrition'] = df_hot['Attrition'].map({'Yes': 1, 'No': 0})

    # Target Encoding para ordinales
    df_pred[cols_ordinales] = df_pred[cols_ordinales].fillna("Unknown")
    df_pred["Attrition"] = np.nan
    df_pred[cols_ordinales] = target_encoder.transform(df_pred[cols_ordinales])

    # Unir DataFrame transformado
    df_pred.drop(columns=cols_nominales, inplace=True)
    df_pred = pd.concat([df_pred, onehot_df], axis=1)

    # Eliminar columna temporal
    df_pred.drop(columns="Attrition", inplace=True)

    # Predicción
    prediccion = modelo.predict(df_pred)
    return prediccion


In [79]:
# Verificar categorías conocidas
for col, categories in zip(cols_nominales, one_hot_encoder.categories_):
    print(f"Columna: {col}")
    print(f"Categorías conocidas: {categories}")

Columna: Education
Categorías conocidas: [1 2 3 4 5]
Columna: Gender
Categorías conocidas: ['Female' 'Male']
Columna: JobRole
Categorías conocidas: ['Healthcare Representative' 'Human Resources' 'Laboratory Technician'
 'Manager' 'Manufacturing Director' 'Research Director'
 'Research Scientist' 'Sales Executive' 'Sales Representative']


In [80]:
# Datos de ejemplo adaptados a las restricciones y valores seleccionables
EnvironmentSatisfaction = 3  # Escala de 1 a 4
WorkLifeBalance = 4          # Escala de 1 a 4
JobInvolvement = 3           # Escala de 1 a 4
JobSatisfaction = 4          # Escala de 1 a 4

# Demografía y datos personales
Age = 35                     # Entre 18 y 60 años
Gender = "Male"              # 'Female' o 'Male'
MaritalStatus = "Married"    # 'Divorced', 'Married', 'Single'
DistanceFromHome = 15        # Entre 1 y 29 km

# Información laboral
Department = "Sales"         # 'Human Resources', 'Research & Development', 'Sales'
JobLevel = 3                 # Escala de 1 a 5
JobRole = "Sales Executive"  # Selección de roles laborales

# Años de experiencia
YearsAtCompany = 10          # Entre 0 y 40 años
YearsSinceLastPromotion = 3  # Entre 0 y 15 años
YearsWithCurrManager = 7     # Entre 0 y 17 años

# Formación y educación
Education = 4                # Escala de 1 a 5
EducationField = "Marketing" # Áreas de educación disponibles
TrainingTimesLastYear = 2    # Entre 0 y 6 veces

# Compensación y beneficios
MonthlyIncome = 50000        # Entre 10,000 y 200,000
PercentSalaryHike = 15       # Entre 11% y 25%
StockOptionLevel = 2         # Entre 0 y 3

# Experiencia laboral
NumCompaniesWorked = 5       # Entre 1 y 60 empresas trabajadas
TotalWorkingYears = 12       # Total de años trabajados
BusinessTravel = "Travel_Rarely"  # Frecuencia de viaje: 'Non-Travel', 'Travel_Frequently', 'Travel_Rarely'

# Evaluación y desempeño
PerformanceRating = 4        # Escala de 3 a 4

In [81]:
prediction = realizar_prediccion(EnvironmentSatisfaction,WorkLifeBalance,JobInvolvement,JobSatisfaction,
                                        Age,Gender,MaritalStatus,DistanceFromHome,Department,JobLevel,JobRole,
                                        YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,Education,
                                        EducationField,TrainingTimesLastYear,MonthlyIncome,PercentSalaryHike,
                                        StockOptionLevel,NumCompaniesWorked,TotalWorkingYears,BusinessTravel,PerformanceRating)

,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating
0,3,4,4,-0.076923,Yes,Travel_Rarely,Sales,0.666667,4,Marketing,Male,3,Sales Executive,Married,0.017164,1.0,0.166667,2,0.222222,2,0.833333,0.666667,0.8,3,4


,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,Attrition,BusinessTravel,Department,DistanceFromHome,EducationField,JobLevel,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,JobInvolvement,PerformanceRating,Education_4,Gender_Male,JobRole_Sales Executive
0,3,4,4,-0.076923,Yes,Travel_Rarely,Sales,0.666667,Marketing,3,Married,0.017164,1.0,0.166667,2,0.222222,2,0.833333,0.666667,0.8,3,4,1.0,1.0,1.0


NameError: name 'df_hot' is not defined